# Howso Engine Intro 

## Overview 

 
Howso Engine is a generalized Machine Learning (ML) and Artificial Intelligence platform that creates powerful decision-making platforms that are fully explainable, auditable, and editable. Howso Engine uses Instance-Based Machine Learning which stores instances, i.e., data points, in memory and makes predictions about new instances given their relationship to existing instances. This technology harnesses a fast spatial query system and techniques from information theory to enhance both performance and accuracy. 

 

Howso Engine is unique in that it utilizes a one-model approach (the data is the model) that enables the user to accomplish: 

 

- "Targetless Learning" - allows the user to predict and characterize any set of variables 

- Supervised Learning 

- Unsupervised Learning 

- Reinforcement Learning 

- Online Learning 

- Discriminative and Generative Modeling 

- Model Interpretation and Explanation 

- Data Imputation 

 

In this notebook we will explore the most basic workflow for training Howso Engine and making predictions. This will enable us to become familiar with basic Howso Engine terminologies, including: 

 

- Action Features 

- Context Features 

- Cases 

- Trainee 

- Train 

- Analyze 

- React 

## Recipe Goals:

This notebook will provide a demonstration of a basic Howso Engine workflow.

In [1]:
from pathlib import Path

import pandas as pd
from pmlb import fetch_data

from howso.engine import (
    load_trainee,
    Trainee,
)
from howso.utilities import infer_feature_attributes

# Step 1: Import Data

Our example dataset for this recipe is the well known `Adult` dataset. This dataset consists of 14 Context Features and 1 Action Feature. The Action Feature in this version of the `Adult` dataset has been renamed to `target` and it takes the form of a binary indicator for whether a person in the data makes more than $50,000/year (*target*=1) or less (*target*=0).

### **Definitions:**

**`Action Feature`:** A feature that the user would like to discriminatively predict or generatively synthesize. In traditional ML these may be referred to as targets, target features, or dependent variables.

**`Context Feature`:** A feature that is used as an input to the model that creates a context from which a prediction is derived. In traditional ML these are often referred to as features, inputs, predictors, or independent features.

In [2]:
df = fetch_data('adult', local_cache_dir="../data/adult")

# Subsample the data to ensure the example runs quickly
df = df.sample(2000)

# Split out the last row for a prediction set and drop the Action Feature
test_case = df.iloc[[-1]].copy()
df.drop(df.index[-1], inplace=True)
test_case = test_case.drop('target', axis=1)

# Step 2: Feature Mapping

Typically, an exploratory analysis is done on the data to get a general feel of the descriptive statistics and data attributes. 

Methods like `describe` from a Pandas dataframe often automatically present these types of information of interest to a user, as shown below. While informative, these descriptive statistics are often used as a sanity check pre- and post-modeling and a model typically doesn't actually use any of these feature attributes.

In [3]:
df.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
count,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000
mean,38.812406,3.869435,185739.513257,10.343672,10.111056,2.638819,6.617309,1.429715,3.673837,0.662331,1141.369685,101.821911,40.873437,36.791896,0.749875
std,13.624204,1.488851,100933.603986,3.865021,2.545614,1.541425,4.231670,1.580961,0.835086,0.473033,7759.456647,429.820657,12.197265,7.699415,0.433193
min,17.000000,0.000000,20308.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,28.000000,4.000000,117554.000000,9.000000,9.000000,2.000000,3.000000,0.000000,4.000000,0.000000,0.000000,0.000000,40.000000,39.000000,0.500000
50%,37.000000,4.000000,175759.000000,11.000000,10.000000,2.000000,7.000000,1.000000,4.000000,1.000000,0.000000,0.000000,40.000000,39.000000,1.000000
75%,48.000000,4.000000,232695.500000,12.000000,12.000000,4.000000,10.000000,3.000000,4.000000,1.000000,0.000000,0.000000,45.000000,39.000000,1.000000
max,90.000000,8.000000,913447.000000,15.000000,16.000000,6.000000,14.000000,5.000000,4.000000,1.000000,99999.000000,3175.000000,99.000000,41.000000,1.000000


In the Howso Engine workflow, feature attributes are an essential part of model building and usage. By incorporating certain feature attributes into the model itself, Howso Engine gains another layer of information that will help in fine-tuning the results. 

In order to assist the user with defining the feature attributes, Howso provides the `infer_feature_attributes` function that automatically processes the dataset for the user.

These feature attributes are based on the existing data, rather than exact descriptive statistics. This is why, for example, the min and max bounds on continuous features are not the exact min and max values of the dataset, but rather an expanded version of those min and max values to allow for some variation. However, users are encouraged to manually adjust feature attributes such as bounds if the properties of any features in the data are well defined.

Additionally, Howso always encourages users to inspect the feature attributes of their data. Incorrect feature attributes can severely impact the accuracy of the model. Specific errors to lookout for are incorrect values for 'type' or 'bounds'.

In [4]:
# Infer features attributes
features = infer_feature_attributes(df)
features

{'age': {'type': 'continuous',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 7.0, 'max': 148.0, 'allow_null': True}},
 'workclass': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 'fnlwgt': {'type': 'continuous',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 8103.0, 'max': 1202604.0, 'allow_null': True}},
 'education': {'type': 'nominal',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'integer', 'size': 8},
  'bounds': {'allow_null': False}},
 'education-num': {'type': 'continuous',
  'data_type': 'number',
  'decimal_places': 0,
  'original_type': {'data_type': 'numeric', 'size': 8},
  'bounds': {'min': 1.0, 'max': 20.0, 'allow_null': True}},
 'marital-status': {'type': 'nominal',
  'dat

To prepare to measure model performance, we also define our action features and context features that we want to use in our experiment.

In [5]:
# Specify Context and Action Features
action_features = ['target']
context_features = features.get_names(without=action_features)

> **Note:** Train Test Split

To gauge model performance, train-test splits are often used in traditional machine learning workflows. Howso Engine does not require the use of train-test split for validation. Please see recipe `6-validation.ipynb` for further explanation. Therefore, we will not use train-test splits our recipes unless the test set serves a specific purpose.

# Step 3: Create Trainee

To begin the Howso Engine workflow, a Trainee is created to act as a base for all of our ML needs. In all subsequent notebooks, we will refer to Howso Engine's model as the Trainee.

### **Definitions:**

**`Trainee`:** A collection of Cases that comprise knowledge. It also includes metadata and parameters. In traditional ML this is referred to as a model.

**`Case`:** A set of feature values representing an observation. In traditional ML, a Case is sometimes referred to as an "observation", "record", or "data point". In database terms, a Case would be a row of values. For supervised learning a Case is a set of Context values and Action values. For unsupervised learning a Case is just a set of feature values. 

In [6]:
# Create the Trainee
t = Trainee(features=features)

Version 24.1.0 of Howso Engine™ is available. You are using version 23.1.1.dev2+gaa28aac.d20240625.
The following parameters from configuration file will override the Amalgam parameters set in the code: {'trace', 'library_path'}


# Step 4: Preprocessing and Training

One benefit of Howso Engine is that most standard forms of data pre-processing such as one-hot encoding and standardization are **NOT** needed, which is in contrast to many modern ML models. This does not include more sophisticated forms of pre-processing such as feature selection or feature engineering, which may still be useful. Fitting is also done in two steps in Howso Engine.

### **Definitions:**

**`Train`:** To add a case or set of cases to the Trainee, making the information contained in these cases available for downstream tasks.

**`Analyze`:** Tune internal parameters to improve performance and accuracy of predictions and metrics. Analysis may be targeted or targetless.  Targetless analysis provides the best balanced set of parameters if an Action Feature is not specified, along with a performance boost while targeted analysis provides a boost to accuracy towards the specified action features. 


In [7]:
# Train
t.train(df)

# Analyze the Trainee
# (By specifying action_features, this becomes a Targeted analysis)
t.analyze(context_features=context_features, action_features=action_features)

# Step 5: Results

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. At this stage in the Howso Engine workflow, a typical use case would be to evaluate the accuracy of the Trainee, which is performed by the `react` method. This is equivalent to `predict` in many traditional Machine Learning workflows, although the `react` method is not solely used for supervised predictions as detailed in subsequent recipes. 

Since we are not using a train-test split, we will use the `react_aggregate` method, which performs a `react` on each of the cases that is trained into the model (or a sample for very large models), then computes the desired metrics. Alternatively, a standard `react` call may be used on untrained data for prediction.

The accuracy is calculated internally as shown in the code below and this is the recommended accuracy metric for nominal action features. Further explanations are available in recipe `6-validation`.

### **Definitions:**

**`React`:** Exposing a Trainee to a new Case's Context Features and an Action Feature for that case is returned. In traditional ML this is often referred to predicting or labeling.

In [11]:
# Recommended metrics
stats = t.react_aggregate(
    # action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["all"]
    }
).target.round(2)


In [ ]:
# The confusion matrix can be retrieved
print("Howso Prediction Results - Confusion Matrix")
matrix = pd.DataFrame(stats["confusion_matrix"]['matrix'])
matrix.index.name = "Predicted"
matrix.columns.name = "Actual"
display(matrix)

Another way to use the `get_prediction_stats` method is to utilize the `condition` parameter. Specifying a condition allows the user to get prediction stats on the subset of cases that matches the specified condition.

For example, we can use the `condition` parameter to find the accuracy, precision, and recall on the subsets of the dataset with each value for Sex.

In [ ]:
# Condition takes a dictionary of feature name to feature value for nominal features
sex_1_stats = t.react_aggregate(
    action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "action_condition": {'sex': 1}
    }
)['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=1 - Accuracy: {sex_1_stats["accuracy"]}, Precision: {sex_1_stats["recall"]}, and Recall: {sex_1_stats["precision"]}')

sex_0_stats = t.react_aggregate(
    action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "action_condition": {'sex': 0}
    }
)['target'].round(2)
print(f'Howso Prediction Results On Cases with Sex=0 - Accuracy: {sex_0_stats["accuracy"]}, Precision: {sex_0_stats["recall"]}, and Recall: {sex_0_stats["precision"]}')

This type of analysis can help users understand the regions of their data that are less predictable, which may provide insight on how data collection can be improved.

# Step 6: Inference

Once Howso Engine is trained and analyzed, it provides the user with a variety of ML capabilities. One of the core capabilities in engine is prediction, which Engine infers a action value based on the given context values. This prediction is a standard output when using `react` and the prediction can be found by navigating the output as shown.

In [ ]:
results = t.react(
    test_case,
    context_features=context_features,
    action_features=action_features,
)

predictions = results['action'][action_features]
predictions

### Predict Function

A `predict` method is also provided as an ease-of-use function that provides the same results as the method shown above using `react` and directly returns the predictions without requiring the user to navigate the output.

In [ ]:
predictions = t.predict(
    test_case,
    context_features=context_features,
    action_features=action_features,
)

predictions

# Step 7: Saving, Loading, and Deleting from your local disk

These methods work for `HowsoDirectClient` and its subclasses.

### Saving and Loading

When saving to disk, a filepath or filename may provided. The filepath provided may be a relative or absolute path. The filepath can include the desired filename with a `.caml` extension. If no filename is at the end of the filepath, the Trainee ID will be used. If just the filename is provided, then the current working directory will be used as the filepath. 

> **Note:** It is recommended, however not necessary that the filename match the trainee ID


### Deleting

When deleting a Trainee, the `delete` method deletes the trainee from the last saved or loaded disk location, as well as memory. If the Trainee has not been saved, `delete` can also be used to just remove a Trainee from memory.

When Trainees are saved, a `.caml` file and a `.txt` version file is saved. The `delete` method assumes that the prefix to the version filename is the same as the `.caml` trainee filename.

In [ ]:
from howso.direct import HowsoDirectClient

# Clients that extend `HowsoDirectClient` will have local file operations
if isinstance(t.client, HowsoDirectClient):
    # Example filepath (same as default)
    cwd = Path.cwd()
    file_path=f"{cwd}/engine_intro_trainee.caml"

    # Saving
    t.save(file_path=file_path)

    # Loading
    t = load_trainee(file_path=file_path)

    # Deleting
    t.delete()

# Conclusion:

Howso Engine provide users with powerful tools to perform a variety of data analysis and prediction tasks. However, making predictions may not suffice if these tools are used for decision-making. As an example with this specific `Adult` dataset, if the user were a bank predicting whether an individual will make over or under $50k in the future for loan decisions, the outcome of prediction efforts might adversely affect the livelihood of that individual as well as the bank if the prediction is inaccurate or biased. Ideally, a predictive tool should provide both accurate results and interpretability so that it can be held accountable for its predictions. Unfortunately, traditional ML techniques often leave a lot to be desired in terms of interpretability. Because of its instance-based ML nature, Howso Engine is inherently interpretable.